In [2]:
import os
import re

from krwordrank.hangle import normalize

In [3]:
project_root = os.path.dirname(os.getcwd())
input_dir = os.path.join(project_root, 'data/raw')
output_dir = os.path.join(project_root, 'data/processed')

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

In [4]:
for filename in os.listdir(input_dir):
    title = ''
    str = ''
    with open(os.path.join(input_dir, filename)) as f:
        title = f.readline().strip()

        #print(f'title: {title}\n')
        #print(f'filename: {filename}\n')

        # join all lines
        str = ''.join(line.strip() for line in f)
        
        # regularize quotation mark
        str = re.sub('[‘’]', "'", str)
        str = re.sub('[“”]', '"', str)

        # remove period in quote
        quote = False
        parenthesis = False
        parenthesis_open = '〈<⟪[(『'
        parenthesis_close = '〉>⟫])』'
        cur_parenthesis = ''
        new_str = ''
        for c in str:
            if re.match('[\"\']', c):
                quote = not quote
            if re.match('[〈\<⟪\[\(『]', c) and cur_parenthesis == '':
                parenthesis = True
                cur_parenthesis = c
                #print('parenthesis on\n')
                continue
            if re.match('[〉\>⟫\]\)』]', c) and parenthesis_open.find(cur_parenthesis) == parenthesis_close.find(c):
                parenthesis = False
                cur_parenthesis = ''
                #print('\nparenthesis off\n')
                continue
            if not parenthesis and (not quote or (quote and not re.match('[\.\?\!]', c))):
                new_str += c
            #if parenthesis:
                #print(c, end='')
        str = new_str
        
        # split sentences
        sents = [normalize(sent) for sent in str.split('.')]
        sents = [sent for sent in sents if sent != '']
        str = '\n'.join(sents)

    with open(os.path.join(output_dir, filename), mode = 'w') as f:
        f.write(f'@title\n{title}\n')
        f.write(f'@content\n{str}')